In [ ]:
from __future__ import print_function

import os
import json
import shutil
import sh
import yaml
from itertools import islice

REPOS_LIST = "/Users/raymondyee/C/src/gitenberg/Second-Folio/list_of_repos.txt"
GITENBERG_DIR = "/Users/raymondyee/C/src/gitenberg/"

METADATA_DIR = "/Users/raymondyee/C/src/gitenberg-dev/giten_site/metadata"
COVERS_DATA = "/Users/raymondyee/C/src/gitenberg/Second-Folio/covers_data.json"

In [ ]:
import os
import glob
import sh
import yaml

from gitenberg import metadata
import jinja2

from second_folio import (GITENBERG_DIR, 
                          all_repos, 
                          apply_to_repos, 
                          travis_template, 
                          latest_epub,
                          new_travis_template,
                          repo_version,
                          )

from github_settings import (username, password)

In [ ]:
repos_with_images = ['Adventures-of-Huckleberry-Finn_76',
'Dracula_345',
'Gulliver-s-Travels_829',
'Jane-Eyre_1260',
'Jude-the-Obscure_153',
'King-Solomon-s-Mines_2166',
'Les-Mis-rables_135']

In [ ]:
repos = repos_with_images[1:]
repos

In [ ]:
list(apply_to_repos(latest_epub, repos=repos))

In [ ]:
import os
import yaml
import pdb

def update_travis_for_locating_images_dir(repo, template, write_updates=False):
    """
    apply https://github.com/gitenberg-dev/templates/commit/c6d1deaaeba617b0f74ce4ed228c3ea6b19a4127
    
    """
    sh.cd(os.path.join(GITENBERG_DIR, repo))

    metadata_path = os.path.join(GITENBERG_DIR, repo, "metadata.yaml")
    travis_path = os.path.join(GITENBERG_DIR, repo, ".travis.yml")
    travis_api_key_path = os.path.join(GITENBERG_DIR, repo, ".travis.deploy.api_key.txt") 

    # read the current metadata file and replace current_ver with next_ver

    (v0, v1, v_updated) = repo_version(repo, version_type='patch', write_version=write_updates)
    if v_updated:
        sh.git.add(metadata_path)
        
    # write new .travis.yml
    (new_template, template_written) = new_travis_template(repo, template, write_template=write_updates)
    if template_written:
        sh.git.add(travis_path)
    
    if write_updates:
        sh.git.commit("-m", "updated .travis.yml to find images directory")
    
    # add tag
    if v_updated:
        sh.git.tag(v1)
        sh.git.push("origin", "master", "--tags")

        return True

    else:
        return False
    

In [ ]:
from itertools import izip

template = template = travis_template()

results = list(apply_to_repos(update_travis_for_locating_images_dir, 
               kwargs={'template':template,
                        'write_updates':True},
               repos=repos))
results

In [ ]:
import requests

def url_status(url):
    r = requests.get(url, allow_redirects=True, stream=True)
    return r.status_code

def repo_epub_status(repo):
    return url_status(latest_epub(repo))

In [ ]:
results = list(izip(repos, apply_to_repos(repo_epub_status, 
               repos=repos)))
results